In [3]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, Dropdown, Button, Output, Checkbox, Label
from IPython.display import display, clear_output
from __future__ import print_function
import json
from IPython.display import display
from tools import simplify_smiles
from tools import load_hdf5_to_dict
from ipywidgets import GridspecLayout

In [4]:
# all data from CCDC
f = open("new_struct2torsion.json")
CCDC_torsions = json.load(f)

f = open("new_struct2angle.json")
CCDC_angles = json.load(f)

f = open("new_struct2bond.json")
CCDC_bonds = json.load(f)

In [5]:
# clean up the CCDC torsion smiles
cleaned_CCDC_torsions = simplify_smiles(CCDC_torsions)
print(f"{len(cleaned_CCDC_torsions)}, {len(CCDC_torsions)}")

14642, 56316


In [6]:
# clean up the CCDC angle smiles
cleaned_CCDC_angles = simplify_smiles(CCDC_angles)
print(f"{len(cleaned_CCDC_angles)}, {len(CCDC_angles)}")

7215, 18179


In [7]:
# clean up the CCDC bond smiles
cleaned_CCDC_bonds = simplify_smiles(CCDC_bonds)
print(f"{len(cleaned_CCDC_bonds)}, {len(CCDC_bonds)}")

2114, 3849


In [8]:
# # Clean up CSD torsions from hdf5
# CCDC_torsions = load_hdf5_to_dict('ccdc_torsion.hdf5')
# cleaned_CCDC_torsions = simplify_smiles(CCDC_torsions)
# print(f"{len(cleaned_CCDC_torsions)}, {len(CCDC_torsions)}")

In [9]:
# # Clean up CSD angles from hdf5
# CCDC_angles = load_hdf5_to_dict('full_ccdc_angle.hdf5')
# cleaned_CCDC_angles = simplify_smiles(CCDC_angles)
# print(f"{len(cleaned_CCDC_angles)}, {len(CCDC_angles)}")

In [10]:
# # Clean up CSD bonds from hdf5
# CCDC_bonds = load_hdf5_to_dict('full_ccdc_bond.hdf5')
# cleaned_CCDC_bonds = simplify_smiles(CCDC_bonds)
# print(f"{len(cleaned_CCDC_bonds)}, {len(CCDC_bonds)}")

In [11]:
# Clean up CSD-H torsions from hdf5
H_torsions = load_hdf5_to_dict('H_torsion.hdf5')
cleaned_CCDC_H_torsions = simplify_smiles(H_torsions)
print(f"{len(cleaned_CCDC_H_torsions)}, {len(H_torsions)}")

4577, 6121


In [12]:
# Clean up CSD-H angles from hdf5
H_angles = load_hdf5_to_dict('H_angle.hdf5')
cleaned_CCDC_H_angles = simplify_smiles(H_angles)
print(f"{len(cleaned_CCDC_H_angles)}, {len(H_angles)}")

2060, 2750


In [13]:
# Clean up CSD-H bonds from hdf5
H_bonds = load_hdf5_to_dict('H_bond.hdf5')
cleaned_CCDC_H_bonds = simplify_smiles(H_bonds)
print(f"{len(cleaned_CCDC_H_bonds)}, {len(H_bonds)}")

659, 856


In [14]:
# Clean up own torsions from hdf5
# own_torsions = load_hdf5_to_dict('owntorsion.hdf5')
# cleaned_own_torsions = simplify_smiles(own_torsions)
# print(f"{len(cleaned_own_torsions)}, {len(own_torsions)}")

In [15]:
# Clean up own angles from hdf5
# own_angles = load_hdf5_to_dict('ownangles.hdf5')
# cleaned_own_angles = simplify_smiles(own_angles)
# print(f"{len(cleaned_own_angles)}, {len(own_angles)}")

In [16]:
# Clean up own bonds from hdf5
# own_bonds = load_hdf5_to_dict('ownbonds.hdf5')
# cleaned_own_bonds = simplify_smiles(own_bonds)
# print(f"{len(cleaned_own_bonds)}, {len(own_bonds)}")

In [17]:
# all own data
f = open("owntorsion.json")
own_torsions = json.load(f)

f = open("ownangle.json")
own_angles = json.load(f)

f = open("ownbond.json")
own_bonds = json.load(f)

In [18]:
# clean up the own torsion smiles
cleaned_own_torsions = simplify_smiles(own_torsions)
print(f"{len(cleaned_own_torsions)}, {len(own_torsions)}")

46, 255


In [19]:
# clean up the own angle smiles
cleaned_own_angles = simplify_smiles(own_angles)
print(f"{len(cleaned_own_angles)}, {len(own_angles)}")

24, 118


In [20]:
# clean up the own bond smiles
cleaned_own_bonds = simplify_smiles(own_bonds)
print(f"{len(cleaned_own_bonds)}, {len(own_bonds)}")

12, 45


Select up to 4 atoms to get internal coordinates

Use 2 atoms to get their bond length, 3 to get the angle and 4 to get the dihedral angle

In [21]:
def adjust_string_for_special_cases(s, special_cases):
    # Replace each special case with a single character (e.g., a marker)
    for case in special_cases:
        s = s.replace(case, "X")  # Assuming "X" does not appear in your string
    return s

def get_adjusted_length(s, special_cases):
    adjusted_s = adjust_string_for_special_cases(s, special_cases)
    return len(adjusted_s)

def select_dictionaries(selected_atoms):
    chosen_dict = {}
    chosen_dict_own = {}

    multi_letter_atoms = ['Cl', 'Br']

    adjusted_length = get_adjusted_length(selected_atoms, multi_letter_atoms)

    # Check the length of selected_atoms
    if adjusted_length == 2:
    # If selected_atoms is 2 atoms, use own_bonds
        chosen_dict_own = cleaned_own_bonds
    elif adjusted_length == 3:
    # If selected_atoms is 3 atoms, use own_angles
        chosen_dict_own = cleaned_own_angles
    elif adjusted_length == 4:
    # If selected_atoms is 4 atoms, use own_torsions
        chosen_dict_own = cleaned_own_torsions
    else:
        print("Invalid number of atoms")   
    # Check if "H" is in selected_atoms
    if "H" in selected_atoms:
    # If "H" is in selected_atoms, use the dictionaries with "H"
        if adjusted_length == 2:
            chosen_dict = cleaned_CCDC_H_bonds
        elif adjusted_length == 3:
            chosen_dict = cleaned_CCDC_H_angles
        elif adjusted_length == 4:
            chosen_dict = cleaned_CCDC_H_torsions
        else:
            print("Invalid number of atoms")
    else:
    # If "H" is not in selected_atoms, use the dictionaries without "H"
        if adjusted_length == 2:
            chosen_dict = cleaned_CCDC_bonds
        elif adjusted_length == 3:
            chosen_dict = cleaned_CCDC_angles
        elif adjusted_length == 4:
            chosen_dict = cleaned_CCDC_torsions
        else:
            print("Invalid number of atoms")
            

    return chosen_dict, chosen_dict_own
    
def draw_plot(selected_atoms, chosen_dict, chosen_dict_own ,fig, index):
    # check the selected_atoms and its reverse in the dictionary
    multi_letter_atoms = ['Cl', 'Br']

    # Function to reverse the order of atoms while keeping multi-letter atoms intact
    def reverse_atoms(atoms):
        result = []
        i = 0
        while i < len(atoms):
            # Check if the current and next character form a multi-letter atom
            if i + 1 < len(atoms) and atoms[i:i+2] in multi_letter_atoms:
                result.insert(0, atoms[i:i+2])
                i += 2
            else:
                result.insert(0, atoms[i])
                i += 1
        return ''.join(result)
    
    
    if selected_atoms not in chosen_dict_own and selected_atoms not in chosen_dict:
        selected_atoms = reverse_atoms(selected_atoms)
      
    # rename lists for better differentiation
    CCDC_data = chosen_dict.get(selected_atoms)
    xyz_data = chosen_dict_own.get(selected_atoms)
    
    if CCDC_data is None or xyz_data is None:
        print(f"{selected_atoms} not found in database!")
        return

    print("Internal Coordinates:", selected_atoms)

    # define counts and bins for CCDC and own data
    counts, bins_CCDC_data = np.histogram(CCDC_data, bins=100, range=(min(CCDC_data*1.1), max(CCDC_data*1.1)))
    counts_CCDC_data = counts / counts.max()

    counts, bins_xyz_data = np.histogram(xyz_data, bins=bins_CCDC_data, range=(min(xyz_data*1.1), max(xyz_data*1.1)))
    counts_xyz_data = counts / counts.max()

    # define your labels
    label_bonds = r"{selected_atoms} Bond Length / Å "
    label_angles = r"{selected_atoms} Angle / $\degree$"
    label_dihedrals = r"{selected_atoms} Dihedral /$\degree$"
    label_ybonds = r"$n_{bond}$ / $n_{max}$"
    label_yangles = r"$n_{angle}$ / $n_{max}$"
    label_ytorsion = r"$n_{dihedral}$ / $n_{max}$"
    title = f"{selected_atoms} Distribution"

    # choose the label based on the chosen dictionary
    if chosen_dict_own is cleaned_own_bonds:
        chosen_label_x = label_bonds.format(selected_atoms=selected_atoms)
        chosen_label_y = label_ybonds
    elif chosen_dict_own is cleaned_own_angles:
        chosen_label_x = label_angles.format(selected_atoms=selected_atoms)
        chosen_label_y = label_yangles
    elif chosen_dict_own is cleaned_own_torsions:
        chosen_label_x = label_dihedrals.format(selected_atoms=selected_atoms)
        chosen_label_y = label_ytorsion
    else:
        print("Invalid dictionary")

    
    # get bar plot normalized to 1
    
    axs = fig.add_subplot(2,2,index)
    axs.bar(bins_CCDC_data[:-1], counts_CCDC_data, width=bins_CCDC_data[1]-bins_CCDC_data[0],color= "skyblue", edgecolor = "black", label="CCDC")
    axs.bar(bins_xyz_data[:-1], counts_xyz_data, width=bins_xyz_data[1]-bins_xyz_data[0], color= "pink", edgecolor = "black", label="Data", alpha=0.5)
    axs.legend(loc="upper left")
    axs.set_xlabel(chosen_label_x)
    axs.set_ylabel(chosen_label_y)
    axs.set_title(title)
    axs.set_ylim(0,1.1)

In [22]:
atoms = ["-", "C", "H", "N", "S", "O", "P", "F", "Cl", "Br", "I"]

def on_clear_click(b):
    for i in range(4):
        if grid[i,grid.n_columns-1] is b:
            for j in range(1, 5):
                grid[i, j].value = "-"   
                
def on_button_click(b):
    
    b.description = "Update"
    
    coordinates1 = [grid[0,x+1].value for x in range(grid.n_columns-2)]
    coordinates2 = [grid[1,x+1].value for x in range(grid.n_columns-2)]
    coordinates3 = [grid[2,x+1].value for x in range(grid.n_columns-2)]
    coordinates4 = [grid[3,x+1].value for x in range(grid.n_columns-2)]

    coordinates1 = ["" if value == "-" else value for value in coordinates1]
    selected_atoms1 = "".join(coordinates1)
    coordinates2 = ["" if value == "-" else value for value in coordinates2]
    selected_atoms2 = "".join(coordinates2)
    coordinates3 = ["" if value == "-" else value for value in coordinates3]
    selected_atoms3 = "".join(coordinates3)
    coordinates4 = ["" if value == "-" else value for value in coordinates4]
    selected_atoms4 = "".join(coordinates4)
    
    with output:
        clear_output()
        fig = plt.figure()
        fig.set_figheight(10)
        fig.set_figwidth(15)
        if selected_atoms1 != "":
            chosen_dict1, chosen_dict_own1 = select_dictionaries(selected_atoms1)
            draw_plot(selected_atoms1, chosen_dict1, chosen_dict_own1 ,fig, 1)
        if selected_atoms2 != "":
            chosen_dict2, chosen_dict_own2 = select_dictionaries(selected_atoms2)
            draw_plot(selected_atoms2, chosen_dict2, chosen_dict_own2, fig, 2)
        if selected_atoms3 != "":
            chosen_dict3, chosen_dict_own3 = select_dictionaries(selected_atoms3)
            draw_plot(selected_atoms3, chosen_dict3, chosen_dict_own3, fig, 3)
        if selected_atoms4 != "":
            chosen_dict4, chosen_dict_own4 = select_dictionaries(selected_atoms4)
            draw_plot(selected_atoms4, chosen_dict4, chosen_dict_own4, fig, 4)
        plt.show()
    
    if check.value:
        internal_coordinates = []
        for ax in fig.axes:
            title = ax.get_title()  # get subplot title
            title_atoms = title.split(' Distribution')[0]  # get the atoms at the start of the title
            internal_coordinates.append(title_atoms)  # add the entire string of atoms to internal_coordinates

        safe_title = "_".join(internal_coordinates)  # create a safe title with only the internal coordinates
        fig.savefig(f"{safe_title}.png")  # save the plot with the title as the filename

grid = GridspecLayout(4, 6, layout=Layout(width="700px"))

for i in range(4):
    for j in range(6):
        if j == 0:
            grid[i, j] = Label(f"Atoms for Plot {i+1}")
        elif j == 5:
            clear_button = Button(description="Clear", layout=Layout(width="100px"))
            clear_button.on_click(on_clear_click)
            grid[i, j] = clear_button
        else:
            grid[i, j] = Dropdown(options=atoms, layout=Layout(width="100px"))
    

display(grid)

check = Checkbox(value=False, description="Save Plots") # checkbox to save the plot
display(check)

output = Output()
display(output)
     
button = Button(description='Start')
button.on_click(on_button_click)
display(button)

GridspecLayout(children=(Label(value='Atoms for Plot 1', layout=Layout(grid_area='widget001')), Dropdown(layou…

Checkbox(value=False, description='Save Plots')

Output()

Button(description='Start', style=ButtonStyle())